# Bar chart race for COVID-19 diagnosed cases from Colombia

Bar chart racer has become a popular data visualization tool to analyze the evolution of a phenomena along the time. In this notebook we'are going to develop a bar chart racer using the package <a href="https://github.com/dexplo/bar_chart_race">bar_chart_race</a> and the data from the <a href="https://www.ins.gov.co/Noticias/Paginas/Coronavirus.aspx">INS</a>.

## 1. Install the required packages
```
pip install pandas
pip install bar_chart_race
pip install ffmpeg
```

## 2. Get the data from INS

The data can be collected using the API of the open data platform <a href="www.datos.gov.co">Datos.gov.co</a>. You can check the details of the endpoint <a href="https://dev.socrata.com/foundry/www.datos.gov.co/gt2j-8ykr">here</a>.

In [1]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.datos.gov.co", None)
              
#Number of cases you want to analyze
n_cases=30000

# First 30000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("gt2j-8ykr",limit=n_cases)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

#Visualization of the first 5 rows of the dataframe
results_df.head()

,atenci_n,ciudad_de_ubicaci_n,codigo_divipola,departamento,edad,estado,fecha_de_muerte,fecha_de_notificaci_n,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,fis,id_de_caso,pa_s_de_procedencia,sexo,tipo
0,Recuperado,Bogotá D.C.,11001,Bogotá D.C.,19,Leve,- -,2020-03-02T00:00:00.000,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,2020-02-27T00:00:00.000,1,Italia,F,Importado
1,Recuperado,Guadalajara de Buga,76111,Valle del Cauca,34,Leve,- -,2020-03-06T00:00:00.000,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,2020-03-04T00:00:00.000,2,España,M,Importado
2,Recuperado,Medellín,5001,Antioquia,50,Leve,- -,2020-03-07T00:00:00.000,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,2020-02-29T00:00:00.000,3,España,F,Importado
3,Recuperado,Medellín,5001,Antioquia,55,Leve,- -,2020-03-09T00:00:00.000,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,2020-03-06T00:00:00.000,4,Colombia,M,Relacionado
4,Recuperado,Medellín,5001,Antioquia,25,Leve,- -,2020-03-09T00:00:00.000,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,2020-03-08T00:00:00.000,5,Colombia,M,Relacionado


## 3. Modification of the dataframe

In this case we want to create a visualization of the positive cases by cities, so we are not going to need all the column. We just need the information of the cities (ciudad_de_ubicaci_n column) and the date of diagnosis (fecha_diagnostico column).

In [2]:
#Filtering the results_df dataframe by the selected columns and create a new dataframe.
mod_df=results_df[['ciudad_de_ubicaci_n','fecha_diagnostico']].copy()

#Rename the columns
mod_df.rename(columns={'fecha_diagnostico': 'date','ciudad_de_ubicaci_n': 'city'},inplace=True)

#Setting the vaues of the column date as datetime
mod_df['date']=pd.to_datetime(mod_df['date'],format='%Y-%m-%d', errors='coerce')

#Create a new column to count by all the values are 1
#This will ease some operations in the next steps
mod_df['count']=1

#Visualization of the first 5 rows of the new dataframe
mod_df.head()

,city,date,count
0,Bogotá D.C.,2020-03-06,1
1,Guadalajara de Buga,2020-03-09,1
2,Medellín,2020-03-09,1
3,Medellín,2020-03-11,1
4,Medellín,2020-03-11,1


## 4. Formatting the dataframe

Now, we need to format the data to the required format for the package <b>bart_chart_race</b>.

### 4.1. Create a pivot table

In [3]:
import numpy as np

#Create a pivot table with mod_df
formmated_df=pd.pivot_table(mod_df, values='count', index=['date'],
                              columns=['city'], aggfunc='count')

#Remove the name of the column axis
formmated_df.rename_axis(None, axis="columns",inplace=True)

#Visualization of the last 5 rows of the formatted dataframe
formmated_df.tail()

,Acacías,Acevedo,Agrado,Aguachica,Aguadas,Agustín Codazzi,Aipe,Albania,Alcalá,Aldana,...,Villeta,Viracachá,Viterbo,Yopal,Yotoco,Yumbo,Zipacón,Zipaquirá,Zona Bananera,Ábrego
date,,,,,,,,,,,,,,,,,,,,,
2020-05-23,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN
2020-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.0,NaN,NaN
2020-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN
2020-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,1.0,3.0,NaN,NaN
2020-05-27,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN


### 4.2 Calculate cummulative cases by date

In the dataframe above we have the number of diagnoesd cases by date, but we need the cummulative cases by date.

In [4]:
#Fill Nan Values
formmated_df.fillna(0, inplace=True)

#Calculate the cummulative cases by date
formmated_df=formmated_df.cumsum()

#Return 0 values to Nan (This is required by the package)
formmated_df.replace(0, np.nan, inplace=True)

#Visualization of the last 5 rows of the formatted dataframe
formmated_df.tail()

,Acacías,Acevedo,Agrado,Aguachica,Aguadas,Agustín Codazzi,Aipe,Albania,Alcalá,Aldana,...,Villeta,Viracachá,Viterbo,Yopal,Yotoco,Yumbo,Zipacón,Zipaquirá,Zona Bananera,Ábrego
date,,,,,,,,,,,,,,,,,,,,,
2020-05-23,15.0,2.0,1.0,2.0,1.0,8.0,4.0,4.0,1.0,1.0,...,5.0,1.0,2.0,23.0,3.0,37.0,NaN,17.0,11.0,1.0
2020-05-24,15.0,2.0,1.0,2.0,1.0,8.0,4.0,4.0,1.0,1.0,...,5.0,1.0,2.0,23.0,3.0,38.0,NaN,20.0,11.0,1.0
2020-05-25,15.0,2.0,1.0,2.0,1.0,8.0,4.0,4.0,1.0,1.0,...,5.0,1.0,2.0,27.0,3.0,38.0,NaN,22.0,11.0,1.0
2020-05-26,15.0,2.0,1.0,2.0,1.0,8.0,4.0,4.0,1.0,1.0,...,5.0,1.0,2.0,27.0,3.0,41.0,1.0,25.0,11.0,1.0
2020-05-27,15.0,2.0,1.0,2.0,1.0,9.0,4.0,4.0,1.0,1.0,...,5.0,1.0,2.0,27.0,3.0,43.0,2.0,25.0,11.0,1.0


## 5. Create the bar chart race!

Now, we have the dataframe in the required format and we can use the <b>bar_chart_race</b> package. 

In [ ]:
#COVID-19 Confirmed Cases in Colombia by Cities
import bar_chart_race as bcr
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    bcr.bar_chart_race(
        df=formmated_df,
        filename='covid19_col_mun.mp4',
        orientation='h',
        sort='desc',
        n_bars=15,
        use_index=True,
        steps_per_period=15,
        period_length=800,
        figsize=(6.5, 3.5),
        cmap='dark24',
        title="COVID-19's confirmed cases in Colombia by cities\nSource: INS",
        bar_label_size=7,
        tick_label_size=7,
        period_label_size=16)

Let's check the result

In [ ]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="covid19_col_mun.mp4" type="video/mp4">
</video></div>